In [1]:
# Library Imports.
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.backends.backend_pdf import PdfPages
import seaborn as sns

# Allows plots to appear directly in the notebook.
%matplotlib inline

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn import tree
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics
from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_score 

import pickle

In [2]:
# Read CSV file into Data Frame:
rt = pd.read_csv('v9_route_39A.csv', keep_default_na=True, delimiter=',', skipinitialspace=True)

In [3]:
rt.shape

(3417494, 27)

## Arrival Difference Column

In [4]:
rt['arr_diff'] = rt['act_stp_arr'] - rt['PLANNEDTIME_ARR']

In [5]:
rt['arr_diff'] = rt['arr_diff'].astype('int64')

In [6]:
rt.head()

,DAYOFSERVICE,TRIPID,PROGRNUMBER,STOPPOINTID,PLANNEDTIME_ARR,VEHICLEID,year,month,day,id,...,stop_actARR_hour,wthr_tr_lt_id,temp,humidity,wind_speed,weather_id,weather_description,dos_trpID_stpID_prgNum,act_stp_arr,arr_diff
0,2018-01-01,5959380,1,767,32400,2172316,2018,1,1,20180101595938001,...,9,2018010109,4.39,86,7.7,802,scattered clouds,2018-01-01-5959380-767-1,32323.0,-77
1,2018-01-01,5959380,2,768,32503,2172316,2018,1,1,20180101595938002,...,9,2018010109,4.39,86,7.7,802,scattered clouds,2018-01-01-5959380-768-2,32461.0,-42
2,2018-01-01,5959380,3,769,32527,2172316,2018,1,1,20180101595938003,...,9,2018010109,4.39,86,7.7,802,scattered clouds,2018-01-01-5959380-769-3,32481.0,-46
3,2018-01-01,5959380,4,770,32565,2172316,2018,1,1,20180101595938004,...,9,2018010109,4.39,86,7.7,802,scattered clouds,2018-01-01-5959380-770-4,32510.0,-55
4,2018-01-01,5959380,5,771,32581,2172316,2018,1,1,20180101595938005,...,9,2018010109,4.39,86,7.7,802,scattered clouds,2018-01-01-5959380-771-5,32520.0,-61


## Hour Column

In [7]:
rt['hour'] = (rt['PLANNEDTIME_ARR'] / 3600).round()

In [8]:
rt['hour'] = rt['hour'].astype('int16')

In [9]:
rt.head()

,DAYOFSERVICE,TRIPID,PROGRNUMBER,STOPPOINTID,PLANNEDTIME_ARR,VEHICLEID,year,month,day,id,...,wthr_tr_lt_id,temp,humidity,wind_speed,weather_id,weather_description,dos_trpID_stpID_prgNum,act_stp_arr,arr_diff,hour
0,2018-01-01,5959380,1,767,32400,2172316,2018,1,1,20180101595938001,...,2018010109,4.39,86,7.7,802,scattered clouds,2018-01-01-5959380-767-1,32323.0,-77,9
1,2018-01-01,5959380,2,768,32503,2172316,2018,1,1,20180101595938002,...,2018010109,4.39,86,7.7,802,scattered clouds,2018-01-01-5959380-768-2,32461.0,-42,9
2,2018-01-01,5959380,3,769,32527,2172316,2018,1,1,20180101595938003,...,2018010109,4.39,86,7.7,802,scattered clouds,2018-01-01-5959380-769-3,32481.0,-46,9
3,2018-01-01,5959380,4,770,32565,2172316,2018,1,1,20180101595938004,...,2018010109,4.39,86,7.7,802,scattered clouds,2018-01-01-5959380-770-4,32510.0,-55,9
4,2018-01-01,5959380,5,771,32581,2172316,2018,1,1,20180101595938005,...,2018010109,4.39,86,7.7,802,scattered clouds,2018-01-01-5959380-771-5,32520.0,-61,9


# Update Rush Hour to use Stop Planned Arrival

In [10]:
# Add Rush Hour Column: 8-9am & 4-6pm (Excludes weekends)
rt['rushHour'] = 0

rt['rushHour'] = np.where(rt['hour'] == 8, 1, rt['rushHour'])

rt['rushHour'] = np.where(rt['hour'] == 16, 1, rt['rushHour'])
rt['rushHour'] = np.where(rt['hour'] == 17, 1, rt['rushHour'])

rt.head()

,DAYOFSERVICE,TRIPID,PROGRNUMBER,STOPPOINTID,PLANNEDTIME_ARR,VEHICLEID,year,month,day,id,...,wthr_tr_lt_id,temp,humidity,wind_speed,weather_id,weather_description,dos_trpID_stpID_prgNum,act_stp_arr,arr_diff,hour
0,2018-01-01,5959380,1,767,32400,2172316,2018,1,1,20180101595938001,...,2018010109,4.39,86,7.7,802,scattered clouds,2018-01-01-5959380-767-1,32323.0,-77,9
1,2018-01-01,5959380,2,768,32503,2172316,2018,1,1,20180101595938002,...,2018010109,4.39,86,7.7,802,scattered clouds,2018-01-01-5959380-768-2,32461.0,-42,9
2,2018-01-01,5959380,3,769,32527,2172316,2018,1,1,20180101595938003,...,2018010109,4.39,86,7.7,802,scattered clouds,2018-01-01-5959380-769-3,32481.0,-46,9
3,2018-01-01,5959380,4,770,32565,2172316,2018,1,1,20180101595938004,...,2018010109,4.39,86,7.7,802,scattered clouds,2018-01-01-5959380-770-4,32510.0,-55,9
4,2018-01-01,5959380,5,771,32581,2172316,2018,1,1,20180101595938005,...,2018010109,4.39,86,7.7,802,scattered clouds,2018-01-01-5959380-771-5,32520.0,-61,9


In [11]:
# Exclude weekends
rt['rushHour'] = np.where(rt['dayOfWeek'] > 4, 0, rt['rushHour'])

## Just 39A Trips going in Direction '1'

In [12]:
rt_39a1 = rt [rt['DIRECTION'] == 1]

In [13]:
rt_39a1.shape

(1585989, 29)

## Shuffling Data

In [14]:
# Row shuffle inspired from Geeks for Geeks: https://www.geeksforgeeks.org/pandas-how-to-shuffle-a-dataframe-rows/
shuf = rt_39a1.sample(frac = 1)
shuf.head()

,DAYOFSERVICE,TRIPID,PROGRNUMBER,STOPPOINTID,PLANNEDTIME_ARR,VEHICLEID,year,month,day,id,...,wthr_tr_lt_id,temp,humidity,wind_speed,weather_id,weather_description,dos_trpID_stpID_prgNum,act_stp_arr,arr_diff,hour
2106708,2018-08-11,7328235,70,2171,45484,2868325,2018,8,11,20180811732823570,...,2018081113,18.76,78,6.2,300,light intensity drizzle,2018-08-11-7328235-2171-70,46107.0,623,13
124775,2018-01-13,6100085,41,1664,37643,2868372,2018,1,13,20180113610008541,...,2018011310,7.39,93,11.8,500,light rain,2018-01-13-6100085-1664-41,37344.0,-299,10
2923867,2018-11-05,8104088,19,786,31425,1000611,2018,11,5,20181105810408819,...,2018110509,11.39,93,4.6,500,light rain,2018-11-05-8104088-786-19,31606.0,181,9
1104343,2018-04-28,6640481,36,1805,29553,2693269,2018,4,28,20180428664048136,...,2018042808,7.39,81,3.6,803,broken clouds,2018-04-28-6640481-1805-36,29466.0,-87,8
2082617,2018-08-09,7324617,4,770,49390,2172311,2018,8,9,20180809732461704,...,2018080914,18.39,45,4.1,803,broken clouds,2018-08-09-7324617-770-4,49614.0,224,14


## Splitting Shuffled Data: Train (70%) & Test (30%)

In [15]:
# train_test_split already includes a shuffle method, but no harm to shuffle again
train, test = train_test_split(shuf, test_size=0.3, random_state=42, shuffle=True)

## Random Forest

## Training

In [16]:
X = train[['month', 'dayOfWeek', 'rushHour', 'hour', 'temp', 'wind_speed']]
y = train.arr_diff

In [17]:
random_forest = RandomForestRegressor(n_estimators = 20, random_state = 42)
random_forest.fit(X, y)

RandomForestRegressor(n_estimators=20, random_state=42)

In [18]:
# Compute the importance of each feature based on the trained random forest regressor
feature_importance = pd.DataFrame({'feature': X.columns, 'importance':random_forest.feature_importances_})
feature_importance.sort_values('importance', ascending=False)

,feature,importance
3,hour,0.244232
4,temp,0.240078
0,month,0.187191
5,wind_speed,0.179052
1,dayOfWeek,0.138831
2,rushHour,0.010615


<h2>Prediction & Evaluation on Training Data</h2>

In [19]:
train_rf_predictions = random_forest.predict(X)

train_actual_vs_pred_rf = pd.concat([y, pd.DataFrame(train_rf_predictions, columns=['Pred_arr_diff'], index=y.index)], axis=1)
train_actual_vs_pred_rf.head(10)

,arr_diff,Pred_arr_diff
2029891,683,364.315039
3020638,1322,545.375203
1794596,334,199.524789
3381931,-19,-141.452273
3015228,124,99.746177
895695,-82,191.787073
3032434,131,422.856362
601647,846,339.889419
949517,493,290.374549
299692,557,221.029877


In [20]:
# Function to output evaluation metrics
def printMetrics(testActualVal, predictions):
    #classification evaluation measures
    print("MAE: ", metrics.mean_absolute_error(testActualVal, predictions))
    print("MSE: ", metrics.mean_squared_error(testActualVal, predictions))
    print("RMSE: ", metrics.mean_squared_error(testActualVal, predictions)**0.5)
    print("R2: ", metrics.r2_score(testActualVal, predictions))

In [21]:
printMetrics(y, train_rf_predictions)

MAE:  296.18499395074366
MSE:  155518.3684083253
RMSE:  394.3581727418937
R2:  0.28251273821683665


<h2>Prediction & Evaluation on Testing Data</h2>

In [22]:
X_test = test[['month', 'dayOfWeek', 'rushHour', 'hour', 'temp', 'wind_speed']]
y_test = test.arr_diff

In [23]:
test_rf_predictions = random_forest.predict(X_test)

test_actual_vs_pred_rf = pd.concat([y_test, pd.DataFrame(test_rf_predictions, columns=['Pred_arr_diff'], index=y_test.index)], axis=1)
test_actual_vs_pred_rf.head(10)

,arr_diff,Pred_arr_diff
3071685,8,30.668579
2271677,-141,-29.233770
2621231,-507,-56.012237
2166821,628,401.827378
440340,215,184.458388
927215,74,455.120499
600543,-31,251.320202
2065690,33,309.369017
183283,-189,85.141936
3315974,612,531.248721


In [24]:
printMetrics(y_test, test_rf_predictions)

MAE:  299.04993419575874
MSE:  158186.58302182853
RMSE:  397.726769305045
R2:  0.2753430205719004


<h2>Prediction & Evaluation on Full Data (5-Fold Cross-Validation):</h2>

In [25]:
X_fold = shuf[['month', 'dayOfWeek', 'rushHour', 'hour', 'temp', 'wind_speed']]
y_fold = shuf.arr_diff

<h3>3-Fold Cross-Validation Metrics:</h3>

In [26]:
mae = cross_val_score(RandomForestRegressor(n_estimators=20, random_state=42), X_fold, y_fold, scoring='neg_mean_absolute_error', cv=3)
mse = cross_val_score(RandomForestRegressor(n_estimators=20, random_state=42), X_fold, y_fold, scoring='neg_mean_squared_error', cv=3)
rmse = cross_val_score(RandomForestRegressor(n_estimators=20, random_state=42), X_fold, y_fold, scoring='neg_root_mean_squared_error', cv=3)
r2 = cross_val_score(RandomForestRegressor(n_estimators=20, random_state=42), X_fold, y_fold, scoring='r2', cv=3)


print("MAE: ", -mae.mean())
print("MSE: ", -mse.mean())
print("RMSE: ", -rmse.mean())
print("R2: ", r2.mean())

MAE:  298.26967994532873
MSE:  157543.97320726418
RMSE:  396.9180813599335
R2:  0.2747109181832685


## Pickle the Model

In [ ]:
# Serialize model object into a file called model.pkl on disk using pickle
with open('39A_1dir_rf_model.pkl', 'wb') as handle:
    pickle.dump(random_forest, handle, pickle.HIGHEST_PROTOCOL)